Hello! This is the code for our 588 project! 

In [82]:
#Get the package we need
import pandas as pd

In [83]:
#Upload Data
data = pd.read_csv('C:/Users/Christopher Cook/Downloads/kaggle/movie_metadata.csv')
list(data)

['color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'gross',
 'genres',
 'actor_1_name',
 'movie_title',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'plot_keywords',
 'movie_imdb_link',
 'num_user_for_reviews',
 'language',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio',
 'movie_facebook_likes']

In [84]:
data = data.dropna(how='any')
data = data.drop(['movie_title','plot_keywords','movie_imdb_link','actor_3_name','actor_2_name','language','genres'],axis=1)
data = pd.get_dummies(data)

In [85]:
data.shape

(3756, 3162)

In [86]:
#Make and standardize Y variable
Y = data['imdb_score']
Y = (Y - Y.mean()) / Y.std()

In [87]:
#Make the X matrix. We need to exclude variables that aren't numbers and the Y variable
X = pd.DataFrame()
for col in list(data):
    if (data[col].dtype == 'float64' or data[col].dtype == 'int64' or data[col].dtype == 'uint8') and (col != 'imdb_score'):
        X[col] = (data[col] - data[col].mean()) / data[col].std()

In [88]:
#Import the Machine learning package and split the data into train and test
from sklearn import linear_model
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,Y)

In [89]:
#declare the model
lasso = linear_model.Lasso(alpha=0.005)

In [90]:
#run the model
lasso.fit(X_train, y_train)

Lasso(alpha=0.005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [91]:
#Score the model
lasso.score(X_test, y_test)

0.37378998801921237

In [92]:
#peek at coefs
lasso.coef_

array([ 0.16411824,  0.13637705,  0.        , ...,  0.0706231 ,
        0.        ,  0.0115226 ])

In [93]:
#for completeness I compare it to a linear regression
linear = linear_model.LinearRegression()
linear.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [94]:
#Score linear regression.
linear.score(X_test,y_test)

-6.3093936891726935e+28

In [95]:
lasso.coef_

array([ 0.16411824,  0.13637705,  0.        , ...,  0.0706231 ,
        0.        ,  0.0115226 ])

In [96]:
#get unpenalized coeficients:
X_newtrain = pd.DataFrame()
X_newtest = pd.DataFrame()
for var in range(0, len(list(X))):
    if lasso.coef_[var] > 0.01 or lasso.coef_[var] <  -0.01:
        
        X_newtrain[list(X)[var]] = X_train[list(X)[var]]
        X_newtest[list(X)[var]] = X_test[list(X)[var]]
print(len(list(X_newtrain)))
linear2 = linear_model.LinearRegression()
linear2.fit(X_newtrain,y_train)
linear2.score(X_newtest,y_test)

596


0.32011000960966429

If you're still interested, here's a toy example that shows what lasso can do 

In [97]:
#create the data
import numpy as np
X1 = pd.DataFrame(np.random.uniform(low=0,high=1,size=(1000,100)))
eps = pd.Series(np.random.uniform(low=0,high=1,size=1000))
Y1 = 1/4 * (X1[0] + X1[1] + X1[2] + eps)
X1_train, X1_test, Y1_train, Y1_test = model_selection.train_test_split(X1,Y1)

In [98]:
#create and fit regressions
lasso1 = linear_model.Lasso(alpha=0.002)
linear1 = linear_model.LinearRegression()
lasso1.fit(X1_train,Y1_train)
linear1.fit(X1_train,Y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [99]:
#score lasso
lasso1.score(X1_test,Y1_test)

0.72738163850099502

In [100]:
#score OLS
linear1.score(X1_test,Y1_test)

0.7367158323902685

As you can see the lasso data did much better on the testing set because it was able to force many of the unimportant coefficients to zero. In fact it looks like only 1 variable outside the true parameters ended up with a non-zero coefficient.

In [101]:
#see lasso params
lasso1.coef_

array([ 0.207106  ,  0.22960653,  0.21969784, -0.        ,  0.        ,
        0.        , -0.        , -0.        ,  0.        , -0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
       -0.        , -0.        ,  0.        , -0.        , -0.        ,
        0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.        , -0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -0.        ,  0.        ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.00111811,  0.        , -0.  

In [104]:
frame = pd.DataFrame()
frame['coef'] = pd.Series(lasso.coef_)

In [105]:
frame['vars'] = pd.Series(list(X))

In [106]:
frame = frame[(frame.coef > 0.07) | (frame.coef < -0.07) ]

In [107]:
frame

,coef,vars
0,0.164118,num_critic_for_reviews
1,0.136377,duration
6,0.403008,num_voted_users
9,-0.105054,num_user_for_reviews
11,-0.221269,title_year
159,-0.085059,director_name_Bob Clark
668,-0.089905,director_name_Jason Friedberg
1618,-0.076696,director_name_Uwe Boll
3147,-0.107699,country_USA
3158,0.070623,content_rating_R


In [108]:
frame.shape

(10, 2)